In [41]:
import torch

In [16]:
text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum. Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam, eaque ipsa quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt explicabo. Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit, sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt. Neque porro quisquam est, qui dolorem ipsum quia dolor sit amet, consectetur, adipisci velit, sed quia non numquam eius modi tempora incidunt ut labore et dolore magnam aliquam quaerat voluptatem. Ut enim ad minima veniam, quis nostrum exercitationem ullam corporis suscipit laboriosam, nisi ut aliquid ex ea commodi consequatur? Quis autem vel eum iure reprehenderit qui in ea voluptate velit esse quam nihil molestiae consequatur, vel illum qui dolorem eum fugiat quo voluptas nulla pariatur?"

zlib

In [17]:
import zlib

# Define the original string
original_string = text

# Compress the string
compressed_data = zlib.compress(original_string.encode('utf-8'))

# Calculate sizes
original_size = len(original_string.encode('utf-8'))
compressed_size = len(compressed_data)

# Print results
print(f"Original size: {original_size} bytes")
print(f"Compressed size: {compressed_size} bytes")

Original size: 1311 bytes
Compressed size: 599 bytes


just-token-number

In [71]:
import sys
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B")

tokens = tokenizer.encode(text)
token_string = "".join([str(token) for token in tokens])
token_size = len(token_string.encode('utf-8'))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


token-rank

In [72]:
tokenizer(" ", return_tensors="pt")

{'input_ids': tensor([[220]]), 'attention_mask': tensor([[1]])}

In [75]:
def find_token_rank(token, preceeding=" "):
    inputs = tokenizer(preceeding, return_tensors="pt")
    logits = model(**inputs).logits
    ranked = torch.argsort(logits[0, -1, :], descending=True)
    rank = torch.where(ranked == token)[0][0].item()
    return rank

rank = find_token_rank(tokens[0])
rank

50918

In [126]:
def find_token_rank(token, preceeding=None):
    if preceeding is None:
        preceeding = tokenizer(" ", return_tensors="pt")
    if isinstance(preceeding, torch.Tensor):
        preceeding = {
            "input_ids": preceeding,
            "attention_mask": torch.ones_like(preceeding)
        }
    if isinstance(preceeding, list):
        preceeding = {
            "input_ids": torch.tensor(preceeding, dtype=torch.long).unsqueeze(0),
            "attention_mask": torch.ones(len(preceeding), dtype=torch.long).unsqueeze(0)
        }
    logits = model(**preceeding).logits
    ranked = torch.argsort(logits[0, -1, :], descending=True)
    rank = torch.where(ranked == token)[0][0].item()
    return rank


text = "hello world 23 css"
tokens = tokenizer.encode(text)
ranks = [find_token_rank(tokens[0])]
for i in range(1, len(tokens)):
    preceeding = tokens[:i]
    rank = find_token_rank(tokens[i], preceeding)
    ranks.append(rank)
print(tokens)
print(ranks)

[14990, 1879, 220, 17, 18, 15877]
[73567, 25, 7, 0, 7, 1919]


In [ ]:
def rank_to_token(rank, preceeding=None):
    if preceeding is None:
        preceeding = tokenizer(" ", return_tensors="pt")
    if isinstance(preceeding, torch.Tensor):
        preceeding = {
            "input_ids": preceeding,
            "attention_mask": torch.ones_like(preceeding)
        }
    if isinstance(preceeding, list):
        preceeding = {
            "input_ids": torch.tensor(preceeding, dtype=torch.long).unsqueeze(0),
            "attention_mask": torch.ones(len(preceeding), dtype=torch.long).unsqueeze(0)
        }
    logits = model(**preceeding).logits
    ranked = torch.argsort(logits[0, -1, :], descending=True)
    token = ranked[rank].item()
    
    return token

decoded = []
decoded.append(rank_to_token(ranks[0]))
for i in range(1, len(ranks)):
    preceeding = tokens[:i]
    decoded.append(rank_to_token(ranks[i], preceeding))
print(decoded)
print(tokenizer.decode(tokens))

[14990, 1879, 220, 17, 18, 15877]
hello world 23 css


In [132]:
def rank_encode(text):
    tokens = tokenizer.encode(text)
    ranks = [find_token_rank(tokens[0])]
    for i in range(1, len(tokens)):
        preceeding = tokens[:i]
        rank = find_token_rank(tokens[i], preceeding)
        ranks.append(rank)
    return ranks

def rank_decode(ranks, return_tokens=False):
    decoded = []
    decoded.append(rank_to_token(ranks[0]))
    for i in range(1, len(ranks)):
        preceeding = decoded
        decoded.append(rank_to_token(ranks[i], preceeding))
    if return_tokens:
        return decoded
    return tokenizer.decode(decoded)

text = "hello world 23 css"
ranks = rank_encode(text)
decoded = rank_decode(ranks)
print(decoded)
print(rank_decode(ranks, return_tokens=True))

hello world 23 css
[14990, 1879, 220, 17, 18, 15877]


In [25]:
print(f"Original size: {original_size} bytes")
print(f"zlib-compressed size: {compressed_size} bytes")
print(f"Just-token-number size: {token_size} bytes")

Original size: 1311 bytes
zlib-compressed size: 599 bytes
Just-token-number size: 1307 bytes
